데이터 수집

In [ ]:
text = [ ['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'] , ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'] ]
부정/긍정 리뷰 텍스트, 1인덱스에 인풋, 2인덱스에 아웃풋 

In [33]:
# 기본 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 다운, import, 리드
! pip install konlpy
import codecs
with codecs.open('data2.txt',encoding='utf-8') as f:
    data = [문장.split('\t') for 문장 in f.read().splitlines()]  # 문장이 150001 개 있는 데이터

# 데이터 모양 확인
data[:10]  # 데이터 모양 확인
data = data[1:]  # 0인덱 필요없으니 제외하고 슬라이스 

# 긍정문장과 부정문장 스플릿
긍정문장_data = [  문장[1] for 문장 in data if 문장[2]=='1'  ] # 1:긍정리뷰 아웃풋     
부정문장_data = [  문장[1] for 문장 in data if 문장[2]=='0'  ] # 0:부정리뷰 아웃풋     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


데이터전처리(단어토큰 -> 형태소 토큰화)

In [ ]:
# 문장을 받으면 '단어/형태소'로 쪼개주는 코드
from konlpy.tag import Okt # 한국어 단어 토크나이저
from tqdm import tqdm # 무한루프에 걸렸는지 진행 중인지 확인하기 위한 # 작업의 진행사항을 알려주는 패키지  ~% 진행 표시
from nltk.util import ngrams # 엔그램 단어 묶기, 문맥
tk = Okt()
def tk_f(문장):
  tk_d=['/'.join(i) for i in tk.pos(문장)] #pos : [ ('단어','형태소') ] 여기서 i에  ( . )튜플들어가고 조인()에들어가면 단어,형태소 사이 쉼표를 /로 조인
  return tk_d
tk_f(긍정문장_data[0]) # ['흠/Noun', '.../Punctuation', '포스터/Noun' ]

In [ ]:
end_l=[]
for 문장 in tqdm(긍정문장_data): 
  tk_data = tk_f(문장)  # tk_f() 는 def로  ['흠/Noun', '.../Punctuation', '포스터/Noun' ] 단어/형태소로 뽑는 함수
  ng = ngrams(tk_data,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol='SE') # ng은 zip형태니까
  # ng출력 = [('SS', '흠/Noun'), ('포스터/Noun', 'SE')]
  end_l+=[t for t in ng] # 리스트형태로 뽑아줌 [('SS', '흠/Noun'), ('포스터/Noun', 'SE')] 타입만 list로 바뀌고 다른 건 같음
end_l  
     

In [39]:
end_l_1=[]
for 문장 in tqdm(부정문장_data):
  tk_data=tk_f(문장)
  bg=ngrams(tk_data,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol='SE')
  end_l_1+=[t for t in bg]

100%|██████████| 75173/75173 [04:48<00:00, 260.50it/s]


In [35]:
# end_l = [('SS', '흠/Noun'), ('포스터/Noun', 'SE')] #긍정문
# end_l_1 = [('SS', '흠/Noun'), ('포스터/Noun', 'SE')] #부정문

from nltk import ConditionalFreqDist 
from nltk.probability import ConditionalProbDist,MLEProbDist
cfd = ConditionalFreqDist(end_l)
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [40]:
cfd1 = ConditionalFreqDist(end_l_1)
cpd1 = ConditionalProbDist(cfd1, MLEProbDist)

In [38]:
#랜덤 문장 생성기
st='SS'
all_l=[]
import random
random.seed(8)
while True:
  st=cpd[st].generate()  # SS다음 나올 단어를 gen으로 그냥 random으로 뽑고 
  if st=="SE": #SE가 뽑힐 때까지 반복 (SS와 SE는 뽑히지 않음)
    break
  d=st.split('/')[0]  # 스플릿후[0]인덱이면 단어를 
  all_l.append(d)  # 리스트에 넣음
n_data=' '.join(all_l) #리스트에 들어간 단어 하나하나를 띄어쓰기 기준으로 조인
n_data # 하니 '영상 진짜 웃기 기도' 한 문장으로 통합됨

'영상 진짜 웃기 기도 하고 이제야 알았다니 떡잎 부터 사소한것하나까지 알고있는 진실 을 훔쳐 보고 다시 한번 쯤 보면 공감 이 넘 좋아요 아 .. 인생 을 감수 하는 대통령 의 코믹 할 영화'

In [ ]:
tk.tagset

In [43]:
def 한글_정리_생성(c="SS",n_seed=8,cpd=cpd): # 'SS' 8 cpd 는 디폴트값
  random.seed(n_seed)
  all_l=[]
  while True:
    w=cpd[c].generate()
    if w=="SE": # SE나오면 브레이크
      break
    w2=w.split('/')[0] # 단어
    pos=w.split('/')[1] # 형태소
    if c=="SS":
      all_l.append(w2.title())
    elif c in ["'","\"",'(']:
      all_l.append(w2)
    elif w in ["'",",",".",")",":",";","?"]:
      all_l.append(w2)
    elif pos in ["Josa", "Punctuation", "Suffix"]:
      all_l.append(w2)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
               "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
      all_l.append(w2)
    else:
      all_l.append(" "+w2)
    c=w
  return ''.join(all_l)
#tf몇번등장햇나 idf

긍정적인 확률 기반 문장생성

In [44]:
한글_정리_생성()

'영상 진짜 웃기 기도하고 이제야 알았다니 떡잎부터 사소한것하나까지 알고있는 진실을 훔쳐 보고 다시 한번쯤 보면 공감이 넘 좋아요 아.. 인생을 감수 하는 대통령의 코믹 할 영화'

부정적인 확률 기반 문장생성

In [45]:
한글_정리_생성(c="SS",n_seed=8,cpd=cpd1)

'망함 그래도 어차피 3 류 영화 귀여워 보니깐 그러지 못한 영화라고는 하지만...'